In [1]:
ref = "C:/Users/Flora/Desktop/M2 Systèmes Complexes/Stage/Bioinfo/ref/"
data = "C:/Users/Flora/Desktop/M2 Systèmes Complexes/Stage/Bioinfo/LTEE-clone-curated/"

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def extract_positions(IS_name):
    '''Generates a file listing the position of insertions that occured at some point in at least
    one of the clones of one of the populations (only one occurrence per position, even if it occured
    in different populations).'''
    
    from os import listdir
    from os.path import isfile, join
    onlyfiles = [f for f in listdir(data) if isfile(join(data, f))] #listing all file names for the .gd files
    files = [] # creating a list that will first contain these names to be called, and later on open .gd files
    file_names = [] # list that will contain names as strings (kept as such for all the analysis)
    readlines_names = [] #list that will contain lists with the lines of each file (obtained via readlines())
    for i in onlyfiles: #fixing an error in the list of files (I don't know why it occurs) and filling the previously defined lists
        if len(i) < 30:
            files.append(i)
            file_names.append(i)
            readlines_names.append(i)
    
    
    for i in range(len(files)):
        files[i] = open(data+files[i],"r") #the list files becomes a list of open files
        readlines_names[i] = files[i].readlines() # the list readlines_names becomes a list of lists, with each list containing 
        #all the lines from one file
        
    insertion_positions = {} # Creating a dictionary to hold the positions of the insertions
    for i in range(len(file_names)): # for each file
        for line in readlines_names[i]: #we look at all the lines in the corresponding readlines_names entry
            line = line.split()
            if 'MOB' in line and IS_name in line: #if the line corresponds to a mutation linked to the IS
                insertion_positions[line[4]] = [line[4], line[6]] #we add an entry for this IS position to our dictionary
                # and save the orientation as well
    
    return insertion_positions
    

In [4]:
def interrupted_genes_proportion(insertion_positions):
    '''Uses the list generated by extract_positions() to extract the type of sequence in which insertions occured.
    Returns the proportion of interrupted sequences that are genes.'''
    
    ref_file = open(ref+'REL606.gff3')
    ref_readlines = ref_file.readlines()
    ref_readlines = ref_readlines[5:8897] # we don't need the 4 first lines or the full fasta sequence
    
    number_insertions_genes = 0
    number_insertions_else = 0
    
    
    #NB: there is a problem in this part, somehow some insertions get counted twice (at the end I have a total of
    # 292 insertions instead of 270 for IS150) and I can't figure out the problem
    for position in list(set(insertion_positions)):
        line_found = False
        for line_ref in ref_readlines:
            if 'CDS' not in line_ref.split()[2]:
                if int(position) >= int(line_ref.split()[3]) and int(position) <= int(line_ref.split()[4]):
                    line_found = True
                    if line_ref.split()[2] == 'gene':
                        number_insertions_genes += 1
                    else:
                        number_insertions_else += 1
        if not line_found:
            number_insertions_else += 1
            
    print('total number of insertions: ', number_insertions_genes + number_insertions_else)
    return number_insertions_genes/(number_insertions_genes + number_insertions_else)

In [5]:
def extract_gene_proportion():
    '''Returns the proportion of coding sequences in the genome of REL606.'''
    
    ref_file = open(ref+'REL606.gff3')
    ref_readlines = ref_file.readlines()
    ref_readlines = ref_readlines[5:8897] # we don't need the 4 first lines or the full fasta sequence
    
    list_lengths = []
    list_types = []

        
    for line in ref_readlines:
        line = line.split()
        if 'CDS' in line[2]:
            sequence_length = int(line[4]) - int(line[3]) # length of the segment described by the line
            sequence_type = line[2]
            list_lengths.append(sequence_length)
            list_types.append(sequence_type)
    
    total_length = sum(list_lengths)
    proportion_genes = 0
    for i in range(len(list_types)):
        if list_types[i] == 'gene':
            proportion_genes += list_lengths[i]/total_length
        
    print('total number of genes: ', len(list_types))
    return proportion_genes
            
    

In [6]:
insertion_positions = extract_positions('IS150')
interrupted_genes_prop = interrupted_genes_proportion(insertion_positions)
proportion_genes_genome = extract_gene_proportion()
print(interrupted_genes_prop)
print(proportion_genes_genome)

total number of insertions:  292
total number of genes:  4564
0.7636986301369864
0.9362800547629626


In [7]:
insertion_positions = extract_positions('IS186')
interrupted_genes_prop = interrupted_genes_proportion(insertion_positions)
proportion_genes_genome = extract_gene_proportion()
print(interrupted_genes_prop)
print(proportion_genes_genome)

total number of insertions:  22
total number of genes:  4564
0.6818181818181818
0.9362800547629626


In [1]:
import scipy.stats as stats

In [5]:
oddsratio, pvalue = stats.fisher_exact([[0.7636986301369864*292, 0.23630136986301364*292],
                                      [0.9362800547629626*4729599, 0.06371994523703739*4729599]])
pvalue

7.507261592935916e-21

In [ ]:
Lg=4729599

In [7]:
oddsratio, pvalue = stats.fisher_exact([[0.6818181818181818*22, 0.31818181818181823*22],
                                      [0.9362800547629626*4729599, 0.06371994523703739*4729599]])
pvalue

0.00022331528047529078

0.31818181818181823